In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/train.csv')
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Our CRAZY machine learning thing :) ...

In [3]:
from nn_model import NNModel
from preprocessing import *
from nn_model import *
import pandas as pd

In [4]:
preprocessing = Preprocessing()
y[y==-1] = 0
y = y.astype(np.int)
preprocessing.replace_outliers_by_nan(tX)

#Delete rows from tX and y, which has more than 8 nan values in tX and 0 label in y
num_nan_in_row = np.isnan(tX).sum(axis=1)
mask = (y==0) & (num_nan_in_row>8)
y = y[~mask]
tX = tX[~mask]

In [89]:
def hyperparamters_tunning(x_train, y_train,
                           x_val, y_val,
                           use_transformations, handling_outliers, degrees,
                           lrs, lambdas, epochs, batch_size):
    best = {
        "models_feat": {},
        "model": None,
        "transformation": None,
        "handling_outlier": None,
        "degree": None,
        "accuracy": 0,
        "lr": None,
        "lambda": None,
    }
    models = {}
    model = None
    units = 1
    activation = 'sigmoid'
    for use_transormation in use_transformations:
        for handling_outlier in handling_outliers:
            for lr in lrs:
                for lambda_ in lambdas:
                    for degree in degrees:
                        preprocessing = Preprocessing(
                            use_transformations=use_transformation,
                            handling_outliers=handling_outlier,
                        )
                        train_data, models = preprocessing.preprocess(
                            x_train,
                            lr=lr,
                            lambda_=lambda_,
                            batch_size=batch_size,
                            epochs=epochs,
                        )
                        
                        model = NNModel(train_data.shape[1])
                        model.add_layer(1, activation='sigmoid')
                        for lr in lrs:
                            for lamda_ in lambdas:
                                model.add_layer(units=units, activation=activation)
                                model.train(train_data, y_train, lr=lr,
                                            lambda_=lambda_,
                                            batch_size=batch_size, epochs=epochs,
                                            verbose=1)
                                preprocessing = Preprocessing(
                                    use_transformations=use_transformation,
                                    handling_outliers = handling_outlier,
                                )
                                val_data, _ = preprocessing.preprocess(
                                    x_val,
                                    train=False,
                                    models=models,
                                    degrees=degree
                                )
                                
                                y_pred = model.predict(val_data)
                                y_pred = y_pred > 0.5
                                y_pred = y_pred.squeeze()
                                accuracy = (y_pred==y_val).mean()
                                if accuracy > best["accuracy"]:
                                    print("Best is updated with accuracy:", accuracy)
                                    best["models_feat"] = models
                                    best["model"] = model
                                    best["transformation"] = use_transformation
                                    best["handling_outlier"] = handling_outlier
                                    best["degree"] = degree
                                    best["accuracy"] = accuracy
                                    best["lambda"] = lambda_
                                    best["lr"] = lr
    return best

In [90]:
num_examples = tX.shape[0]
val_size = 10000
train_samples = np.random.choice(num_examples, replace=False, size=val_size)
mask_val = np.isin(np.arange(tX.shape[0]), train_samples)
x_val, y_val = tX[mask_val], y[mask_val]
x_train, y_train = tX[~mask_val], y[~mask_val]

In [91]:
use_transformation = [True, False]
handling_outliers = ['predict', 'fill_mean', 'remove']
lrs = [0.003, 0.01, 0.03, 0.1]
lambdas = [0.3, 1, 10, 100]
degrees = [np.arange(2, 4), np.arange(2, 5)]
epochs = 1000
batch_size = 64

In [99]:
use_transformation = [True]
handling_outliers = ['predict']
lrs = [0.03]
lambdas = [100]
degrees = [np.arange(2, 4)]
epochs = 10
batch_size = 64

In [100]:
best = hyperparamters_tunning(x_train, y_train,
                              x_val, y_val,
                              use_transformations, handling_outliers, degrees,
                              lrs, lambdas, epochs, batch_size)

Preprocesing started!

Training started
>Epoch #10:	[####################]; Loss: 0.4996845798801224
Training ended

Training started
>Epoch #10:	[####################]; Loss: 0.49442000042403716
Training ended

Training started
>Epoch #10:	[####################]; Loss: 0.4974290313376362
Training ended

Training started
>Epoch #10:	[####################]; Loss: 0.5011129731083889
Training ended

Training started
>Epoch #10:	[####################]; Loss: 0.4900865163739426
Training ended

Training started
>Epoch #10:	[####################]; Loss: 0.5001987195599065
Training ended

Training started
>Epoch #10:	[####################]; Loss: 0.4786226205242275
Training ended

Training started
>Epoch #10:	[####################]; Loss: 0.5044487361201369
Training ended

Training started
>Epoch #10:	[####################]; Loss: 0.5015217962625237
Training ended

Training started
>Epoch #10:	[####################]; Loss: 0.4829073808215396
Training ended

Training started
>Epoch #10:	[######

In [101]:
model = best["model"]
best["models_feat"] = models
use_transformation = best["transformation"]
handling_outlier = best["handling_outlier"]
degree = best["degree"]
accuracy = best["accuracy"]
lambda_ = best["lambda"]
lr = best["lr"]

In [102]:
pred = model.predict(train_data)

In [103]:
res = pred>0.5

In [104]:
res = res.squeeze()

In [105]:
pred = np.zeros(y.shape)

In [106]:
pred[res] = 1

In [107]:
np.mean(pred==y)

0.5120885755130169

## Generate predictions and save ouput in csv format for submission:

In [108]:
DATA_TEST_PATH = DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/test.csv')
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [109]:
preprocessing = Preprocessing()
test_data, models = preprocessing.preprocess(tX_test, train=False, models=models)

Preprocesing started!

Preprocessing ended



In [22]:
OUTPUT_PATH = 'prediction' # TODO: fill in desired name of output file for submission
y_pred = model.predict(test_data)
res = y_pred>0.5
res = res.squeeze()
pred = -np.ones(res.shape)
pred[res] = 1
create_csv_submission(ids_test, pred, OUTPUT_PATH)